# 1 - Run Struphy main file in a notebook

This tutorial is about the Struphy main execution file `struphy/main.py`. The file is executed from the console upon calling
```
    struphy run MODEL
```
Please visit https://struphy.pages.mpcdf.de/struphy/sections/userguide.html for detailed information about this command. In this tutorial, we shall

1. Import `struphy/main.py` and look at its functionality.
2. Create some default prameter files and change some parameters.
3. Understand the normalization of Struphy models (which units are used).
3. Run the model [LinearMHDVlasovCC](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.hybrid.LinearMHDVlasovCC) in the notebook (without invoking the console).

## Main execution file and parameters

In [ ]:
from struphy.main import main

help(main)

The function `main.py` has three mandatory arguments:

- `model_name`
- `parameters`
- `path_out`

Available Struphy models for `model_name` can be listed from the console:

In [ ]:
!{'struphy run -h'}

In Struphy, parameters are passed to a model via a dictionary that is stored in `.yml` format (the "parameter file") in the current input path. The current I/O paths can be obtained via

In [ ]:
!{'struphy -p'}

Let us change the current I/O paths to the default:

In [ ]:
!{'struphy --set-iob d'}

In the default input path, there is one "core" parameter file `parameters.yml` that contains all 9 possible top-level keys mentioned in the [Struphy userguide](https://struphy.pages.mpcdf.de/struphy/sections/userguide.html#setting-simulation-parameters):

In [ ]:
import os
import struphy
import yaml

core_name = os.path.join(struphy.__path__[0], 'io/inp', 'parameters.yml')

with open(core_name) as file:
    core_params = yaml.load(file, Loader=yaml.FullLoader)
    
for key, val in core_params.items():
    print(key, ':')
    print(val, '\n')

Model-specific parameter files must be created from the core file via the console. For example,

In [ ]:
!{'struphy params Maxwell -y'}

leads to the creation of `params_Maxwell.yml` in the current input path:

In [ ]:
pfile = os.path.join(struphy.__path__[0], 'io/inp', 'params_Maxwell.yml')

with open(pfile) as file:
    params_Maxwell = yaml.load(file, Loader=yaml.FullLoader)
    
for key, val in params_Maxwell.items():
    print(key, ':')
    print(val, '\n')

Each model has its specific structure of the parameter file, created from the core file with the command `struphy params MODEL`:

In [ ]:
!{'struphy params -h'}

 The model parameter files can differ in top-level keys, variable names and the `options` available to each species. The available options for a model are accessible via the corresponding class method, for example:

In [ ]:
from struphy.models.toy import Maxwell
for key, val in Maxwell.options().items():
    print(key, ':')
    print(val, '\n')

From the command line, this information can be accessed via

In [ ]:
!{'struphy params Maxwell --options'}

The possible choices of the options are given as lists within the options dictionary ; the first list entry is the default value.  The key structure of the options dictionary is exactly the same as it appears in the parameter file:

In [ ]:
print(params_Maxwell['em_fields']['options']['solver'])


In this tutorial, we shall simulate the current coupling hybrid model [LinearMHDVlasovCC](https://struphy.pages.mpcdf.de/struphy/sections/models.html#struphy.models.hybrid.LinearMHDVlasovCC). Let us set the `model_name` argument for the `main` file, look at the options of this model and create the default parameter file:

In [ ]:
model_name = 'LinearMHDVlasovCC'

from struphy.models.hybrid import LinearMHDVlasovCC
for key, val in LinearMHDVlasovCC.options().items():
    print(key, ':')
    print(val, '\n')

In [ ]:
!{'struphy params LinearMHDVlasovCC -y --file tutor_01.yml'}

In [ ]:
pfile = os.path.join(struphy.__path__[0], 'io/inp', 'tutor_01.yml')

with open(pfile) as file:
    parameters = yaml.load(file, Loader=yaml.FullLoader)
    
for key, val in parameters.items():
    print(key, ':')
    print(val, '\n')

## Struphy normalization (units)

In this section we shall gain an understanding of the units used in Struphy (model normalization). 

In the present example, the geometry is a `Cuboid` with default parameters:

In [ ]:
from struphy.geometry.domains import Cuboid

cube = Cuboid()

cube.params_map

The side length in direction `i = 1,2,3` is given by `ri - li`. The question arises in which unit of length these numbers are expressed (meters, millimeters, light years?). From the console, the units can be checked via

In [ ]:
!{'struphy units LinearMHDVlasovCC -i tutor_01.yml'}

Here, two informations are passed to `struphy units`, namely the model name (`LinearMHDVlasovCC`) and the parameter file (`tutor_01.yml`). The former is obvious because each Struphy model has its own specific normalization, stated in the model's documentation (and docstring). The latter, however, is not obvious (parameters influence the units?). **Indeed, Struphy provides the flexibility that the units of each model can be influenced by the user via the parameter file.**

Let us check the relevant section in the dictionary:

In [ ]:
parameters['units']

Here, the user can set

1. the unit of length $\hat x$ in meter
2. the unit of the magnetic field strength $\hat B$ in Tesla
3. the unit of the number density $\hat n$ in $10^{20}$ $m^{-3}$.

In the above example we have $\hat x = 1\,m$, $\hat B = 1\,T$ and $\hat n = 10^{20}$ $m^{-3}$. 

 All other units, such as for velocity $\hat v$ or time $\hat t$ etc., are derived from the three basic units above. How is this achieved? In Struphy, each model has the two class methods
 
- `velocity_scale`
- `bulk_species`

These have been set by the model developer (hard-coded) and cannot be changed by the user. They determine the derived units in the following way:

The `bulk_species` sets the mass number ($A$) and charge number ($Z$) to be used in the calculation of units:


In [ ]:
from struphy.models.hybrid import LinearMHDVlasovCC

print(LinearMHDVlasovCC.bulk_species())

parameters['fluid'][LinearMHDVlasovCC.bulk_species()]['phys_params']


The `velocity_scale` (partly) determines the velocity unit $\hat v$. It has been set by the model developer to one of the following:

1. speed of light, $\hat v = c$
2. Alfvén speed of the bulk species, $\hat v = v_\textnormal{A, bulk} = \sqrt{\hat B^2 / (m_\textnormal{bulk} \hat n \mu_0)}$
3. Cyclotron speed of the bulk species, $\hat v = \hat x \Omega_\textnormal{c, bulk}/(2\pi) = \hat x\, q_\textnormal{bulk} \hat B /(m_\textnormal{bulk}2\pi)$

In [ ]:
print(LinearMHDVlasovCC.velocity_scale())


The three possible velocities scales are entirely defined in terms of:

- the three units $\hat x$, $\hat B$, $\hat n$, which are provided by the user (who can thus also influence $\hat v$)
- the `bulk_species` (through $m_\textnormal{bulk} = m_\textnormal{proton} A$ and $q_\textnormal{bulk} = q_\textnormal{e}Z$). 

The associated time scale is then automatically given by
$$
 \hat t = \hat x / \hat v \,.
$$

 To summarize: qualitatively, the `velocity_scale` and the `bulk_species` are fixed within each model by the developer (hard-coded). Quantitatively, the values (here for the Alfvén speed and the MHD charge and mass) are set by the user through the parameter file. 

Please check out https://struphy.pages.mpcdf.de/struphy/sections/models.html#normalization for further discussion on the units used in Struphy. In this tutorial, instead of the console, we can inspect the units of our run also directly in this notebook:

In [ ]:
units, equation_params = LinearMHDVlasovCC.model_units(parameters, verbose=True)
units

Aside from the units, there are also the `equation_params` returned, namely

- `alpha` being the ratio of the unit plasma frequency to the unit cyclotron frequency
- `epsilon` being the inverse of the unit cyclotron frequency times the time unit

for each species.

In [1]:
equation_params

NameError: name 'equation_params' is not defined

## Run Struphy main

Let us get back to the parameter file and change some entries in the parameter dictionary before we run the model:

In [ ]:
parameters['grid']['Nel'] = [18, 14, 4]
parameters['kinetic']['energetic_ions']['phys_params']['A'] = 4.

We are now ready to call the Struphy main file. A tutorial of how to post-process the generated simulation data is available [here](https://struphy.pages.mpcdf.de/struphy/doc/_build/html/tutorials/tutorial_02_postproc_standard_plotting.html). 

The simulation results will be stored in the default output path under the folder `tutorial_01/`:


In [ ]:
path_out = os.path.join(struphy.__path__[0], 'io/out', 'tutorial_01')

Let us finally call `main`:

In [ ]:
main(model_name, parameters, path_out)

The previous call was equivalent to the console command
```
struphy run LinearMHDVlasovCC -i tutor_01.yml -o tutorial_01
```